In [1]:
import os
os.environ.setdefault('AWS_PROFILE', 'gh-activity')

'gh-activity'

In [2]:
# check existing buckets
!aws s3 ls

2022-11-17 15:06:21 local2s34lambda


In [3]:
# Creates buckets if they don't exist.
# throw an error if existing buckets or name already taken
!aws s3 mb s3://local2s34lambda

make_bucket failed: s3://local2s34lambda An error occurred (BucketAlreadyOwnedByYou) when calling the CreateBucket operation: Your previous request to create the named bucket succeeded and you already own it.


In [4]:
# remove the content of the buckets
!aws s3 rm s3://local2s34lambda/ --recursive

delete: s3://local2s34lambda/data/s
delete: s3://local2s34lambda/data/2022-06-05-0.json.gz


In [5]:
import boto3

dynamodb = boto3.resource('dynamodb') # selecting service
table = dynamodb.Table('jobs') # selecting table
table.delete_item(Key={'job_id': 'ghactivity_ingest'}) # delete item
item = {
    'job_id': 'ghactivity_ingest',
    'job_description': 'Ingest ghactivity data to s3',
    'is_active': 'Y',
    'baseline_days': 3
}
table.put_item(Item=item) # inserting items

{'ResponseMetadata': {'RequestId': '1111EQ1JN18DTLB0CISJECDURJVV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Fri, 18 Nov 2022 09:12:17 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': '1111EQ1JN18DTLB0CISJECDURJVV4KQNSO5AEMVJF66Q9ASUAAJG',
   'x-amz-crc32': '2745614147'},
  'RetryAttempts': 0}}

In [6]:
dynamodb = boto3.resource('dynamodb')
table = dynamodb.Table('jobs')

# extracting particular entry inside the table
baseline_days = table. \
    get_item(Key={'job_id': 'ghactivity_ingest'})['Item']['baseline_days']

In [7]:
baseline_days

Decimal('3')

In [8]:
int(baseline_days)

3

In [9]:
from datetime import datetime as dt
from datetime import timedelta as td

next_file = f'{dt.strftime(dt.now().date() - td(days=int(baseline_days)), "%Y-%m-%d")}-0.json.gz'

In [10]:
next_file

'2022-11-15-0.json.gz'

In [11]:
import requests
res = requests.get(f'https://data.gharchive.org/{next_file}')

In [12]:
s3_client = boto3.client('s3')

In [13]:
type(s3_client)

botocore.client.S3

In [16]:
s3_client.put_object?

Signature: s3_client.put_object(*args, **kwargs)
Docstring:
Adds an object to a bucket. You must have WRITE permissions on a bucket to add an object to it.

 

Amazon S3 never adds partial objects; if you receive a success response, Amazon S3 added the entire object to the bucket.

 

Amazon S3 is a distributed system. If it receives multiple write requests for the same object simultaneously, it overwrites all but the last object written. Amazon S3 does not provide object locking; if you need this, make sure to build it into your application layer or use versioning instead.

 

To ensure that data is not corrupted traversing the network, use the ``Content-MD5`` header. When you use this header, Amazon S3 checks the object against the provided MD5 value and, if they do not match, returns an error. Additionally, you can calculate the MD5 while putting an object to Amazon S3 and compare the returned ETag to the calculated MD5 value.

 

.. note::

   

   
  * To successfully complete the

As per documentation, body must be in bytes format

In [17]:
type(res.content)

bytes

In [18]:
bucket_name = 'local2s34lambda'

In [19]:
upload_res = s3_client.put_object(
   Bucket=bucket_name,
   Key=f'landing/gh-activity/{next_file}',
   Body=res.content
)

In [20]:
upload_res

{'ResponseMetadata': {'RequestId': 'R8ERPW8K42TYB907',
  'HostId': 'K1ZG9H8Xc4yF2tByA6ufsOdOKi9ySIPlRDogI3EoUi3FvQ9DGY84/BKR0xBChzObQXRdH2grED4=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'K1ZG9H8Xc4yF2tByA6ufsOdOKi9ySIPlRDogI3EoUi3FvQ9DGY84/BKR0xBChzObQXRdH2grED4=',
   'x-amz-request-id': 'R8ERPW8K42TYB907',
   'date': 'Fri, 18 Nov 2022 09:18:13 GMT',
   'etag': '"950dff9a89070b1ae05b19844c5ed6f1"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"950dff9a89070b1ae05b19844c5ed6f1"'}

In [22]:
# lets cross-check whether the object exists in S3 or not
!aws s3 ls s3://local2s34lambda/landing/gh-activity/

2022-11-18 14:48:13   97433535 2022-11-15-0.json.gz


In [23]:
import pandas as pd

In [25]:
# lets check whether we can read the object from the S3 bucket
df = pd.read_json(
    f's3://{bucket_name}/landing/gh-activity/{next_file}',
    lines=True,
    orient='records'
)

In [26]:
df.shape

(159073, 8)

In [27]:
df.count()

id            159073
type          159073
actor         159073
repo          159073
payload       159073
public        159073
created_at    159073
org            45201
dtype: int64

In [28]:
df

,id,type,actor,repo,payload,public,created_at,org
0,25235256271,PushEvent,"{'id': 96150274, 'login': 'Mateocontrerass', '...","{'id': 555064516, 'name': 'Mateocontrerass/Pri...","{'push_id': 11661920017, 'size': 1, 'distinct_...",True,2022-11-15 00:00:00+00:00,NaN
1,25235256276,PushEvent,"{'id': 33708658, 'login': 'danieljhkim', 'disp...","{'id': 498571960, 'name': 'coinStatData/CoinSt...","{'push_id': 11661920015, 'size': 2, 'distinct_...",True,2022-11-15 00:00:00+00:00,"{'id': 106642051, 'login': 'coinStatData', 'gr..."
2,25235256284,PullRequestReviewEvent,"{'id': 41898282, 'login': 'github-actions[bot]...","{'id': 204516692, 'name': 'lowRISC/opentitan',...","{'action': 'created', 'review': {'id': 1180011...",True,2022-11-15 00:00:00+00:00,"{'id': 7814611, 'login': 'lowRISC', 'gravatar_..."
3,25235256286,PushEvent,"{'id': 79616833, 'login': 'gutenfries', 'displ...","{'id': 508087807, 'name': 'gutenfries/gutenfri...","{'push_id': 11661920018, 'size': 1, 'distinct_...",True,2022-11-15 00:00:00+00:00,NaN
4,25235256287,WatchEvent,"{'id': 35374714, 'login': 'davidycliao', 'disp...","{'id': 562271532, 'name': 'schochastics/rtoot'...",{'action': 'started'},True,2022-11-15 00:00:00+00:00,NaN
...,...,...,...,...,...,...,...,...
159068,25236024065,PullRequestReviewCommentEvent,"{'id': 6237050, 'login': 'ebyhr', 'display_log...","{'id': 166515022, 'name': 'trinodb/trino', 'ur...","{'action': 'created', 'comment': {'url': 'http...",True,2022-11-15 00:53:49+00:00,"{'id': 34147222, 'login': 'trinodb', 'gravatar..."
159069,25236024751,PullRequestReviewCommentEvent,"{'id': 5150259, 'login': 'husobee', 'display_l...","{'id': 137265074, 'name': 'brave-intl/challeng...","{'action': 'created', 'comment': {'url': 'http...",True,2022-11-15 00:41:38+00:00,"{'id': 28767864, 'login': 'brave-intl', 'grava..."
159070,25236024769,PullRequestReviewCommentEvent,"{'id': 5150259, 'login': 'husobee', 'display_l...","{'id': 137265074, 'name': 'brave-intl/challeng...","{'action': 'created', 'comment': {'url': 'http...",True,2022-11-15 00:44:27+00:00,"{'id': 28767864, 'login': 'brave-intl', 'grava..."
159071,25236024772,PullRequestReviewCommentEvent,"{'id': 5150259, 'login': 'husobee', 'display_l...","{'id': 137265074, 'name': 'brave-intl/challeng...","{'action': 'created', 'comment': {'url': 'http...",True,2022-11-15 00:49:28+00:00,"{'id': 28767864, 'login': 'brave-intl', 'grava..."
